In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Imports and load data

In [11]:
import pandas as pd
import lightgbm as lgb
from sklearn.metrics import mean_absolute_error
from datetime import datetime
import mlflow
import os
from dotenv import load_dotenv
import matplotlib.pyplot as plt

# --- Load environment variables and MLflow setup ---
load_dotenv()
mlflow.set_tracking_uri(os.getenv("MLFLOW_TRACKING_URI"))
mlflow.set_experiment("citibike_full_lag_lgbm_hyper")

<Experiment: artifact_location='mlflow-artifacts:/38e47cf353fc429e90519455ae96b9da', creation_time=1746878000534, experiment_id='3', last_update_time=1746878000534, lifecycle_stage='active', name='citibike_full_lag_lgbm_hyper', tags={}>

# Load full lag data

In [12]:
df = pd.read_parquet("../data/processed/2023/citibike_lag672_top3.parquet")
df['start_hour'] = pd.to_datetime(df['start_hour'])
print(df.shape)
df.head()


(24264, 687)


,start_station_id,start_hour,rides,hour,dow,doy,sin_hour,cos_hour,sin_dow,cos_dow,...,lag_662,lag_663,lag_664,lag_665,lag_666,lag_667,lag_668,lag_669,lag_670,lag_671
0,5329.03,2023-01-29 00:00:00,0,0,6,29,0.000000,1.000000,-0.781831,0.62349,...,15.0,9.0,7.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
1,5329.03,2023-01-29 01:00:00,0,1,6,29,0.258819,0.965926,-0.781831,0.62349,...,12.0,15.0,9.0,7.0,1.0,1.0,0.0,0.0,1.0,0.0
2,5329.03,2023-01-29 02:00:00,1,2,6,29,0.500000,0.866025,-0.781831,0.62349,...,10.0,12.0,15.0,9.0,7.0,1.0,1.0,0.0,0.0,1.0
3,5329.03,2023-01-29 03:00:00,0,3,6,29,0.707107,0.707107,-0.781831,0.62349,...,25.0,10.0,12.0,15.0,9.0,7.0,1.0,1.0,0.0,0.0
4,5329.03,2023-01-29 04:00:00,0,4,6,29,0.866025,0.500000,-0.781831,0.62349,...,23.0,25.0,10.0,12.0,15.0,9.0,7.0,1.0,1.0,0.0


# Drop duplicates if any

In [13]:
num_duplicates = df.duplicated(subset=["start_station_id", "start_hour"]).sum()
print(f"🔍 Duplicate rows (by start_station_id + start_hour): {num_duplicates}")
df = df.drop_duplicates(subset=["start_station_id", "start_hour"])

🔍 Duplicate rows (by start_station_id + start_hour): 0


# Feature and target cols

In [14]:
# --- Feature & target columns ---
feature_cols = [col for col in df.columns if col.startswith("lag_")]
target_col = "target_t_plus_1"

# Time-based split: train = before Dec 2023, test = Dec onwards

In [15]:
cutoff = datetime(2023, 12, 1)
train_df = df[df["start_hour"] < cutoff].copy()
test_df = df[df["start_hour"] >= cutoff].copy()

# Drop rows with missing values *after* splitting

In [16]:
# Drop rows with missing values before splitting
drop_cols = feature_cols + [target_col]
train_df = train_df.dropna(subset=drop_cols)
test_df = test_df.dropna(subset=drop_cols)

X_train = train_df[feature_cols]
y_train = train_df[target_col]
X_test = test_df[feature_cols]
y_test = test_df[target_col]

In [17]:

print("Shapes after time-based split and NaN drop:")
print("X_train:", X_train.shape, "| y_train:", y_train.shape)
print("X_test:", X_test.shape, "| y_test:", y_test.shape)

Shapes after time-based split and NaN drop:
X_train: (22032, 672) | y_train: (22032,)
X_test: (2229, 672) | y_test: (2229,)


# Hyperparameter tuning with optuna

In [18]:
# --- Hyperparameter tuning with Optuna ---
import optuna

def objective(trial):
    params = {
        "objective": "regression_l1",
        "n_estimators": trial.suggest_int("n_estimators", 100, 1000),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 20, 3000),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "random_state": 42,
    }
    model = lgb.LGBMRegressor(**params)
    model.fit(
        X_train,
        y_train,
        eval_set=[(X_test, y_test)],
        eval_metric="mae",
        callbacks=[lgb.early_stopping(10), lgb.log_evaluation(0)]
    ), lgb.log_evaluation(0)  # Add early stopping and suppress log output
    preds = model.predict(X_test)
    return mean_absolute_error(y_test, preds)

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=75)

print("Best trial:")
print(study.best_trial)


[I 2025-05-10 08:04:03,837] A new study created in memory with name: no-name-40c3be6e-c2ac-4315-8a1b-267a08403482


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.021365 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 64448
[LightGBM] [Info] Number of data points in the train set: 22032, number of used features: 672
[LightGBM] [Info] Start training from score 10.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 10 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain:

[I 2025-05-10 08:04:08,239] Trial 0 finished with value: 3.0170057029529906 and parameters: {'n_estimators': 283, 'learning_rate': 0.05670944360673048, 'num_leaves': 411, 'max_depth': 8, 'min_child_samples': 42, 'subsample': 0.7843913214764097, 'colsample_bytree': 0.9269722389077256}. Best is trial 0 with value: 3.0170057029529906.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[I 2025-05-10 08:04:34,796] Trial 1 finished with value: 3.0942658330987873 and parameters: {'n_estimators': 684, 'learning_rate': 0.023670153008831116, 'num_leaves': 645, 'max_depth': 15, 'min_child_samples': 7, 'subsample': 0.514051471655506, 'colsample_bytree': 0.6131600099061651}. Best is trial 0 with value: 3.0170057029529906.


...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Early stopping, best iteration is:
[604]	valid_0's l1: 2.96158
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.023042 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 64448
[LightGBM] [Info] Number of data points in the train set: 22032, number of used features: 672
[LightGBM] [Info] Start training from score 10.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 10 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Light

[I 2025-05-10 08:18:46,418] Trial 73 finished with value: 2.9614567135822623 and parameters: {'n_estimators': 974, 'learning_rate': 0.012209844309469661, 'num_leaves': 746, 'max_depth': 13, 'min_child_samples': 94, 'subsample': 0.8424425522295729, 'colsample_bytree': 0.6335880403712519}. Best is trial 32 with value: 2.951322687768928.


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.029387 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 64448
[LightGBM] [Info] Number of data points in the train set: 22032, number of used features: 672
[LightGBM] [Info] Start training from score 10.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 10 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain:

[I 2025-05-10 08:19:03,996] Trial 74 finished with value: 2.961148350498035 and parameters: {'n_estimators': 986, 'learning_rate': 0.012979138856663967, 'num_leaves': 730, 'max_depth': 14, 'min_child_samples': 94, 'subsample': 0.8380454392171612, 'colsample_bytree': 0.6355545463883058}. Best is trial 32 with value: 2.951322687768928.


Best trial:
FrozenTrial(number=32, state=1, values=[2.951322687768928], datetime_start=datetime.datetime(2025, 5, 10, 8, 8, 13, 134114), datetime_complete=datetime.datetime(2025, 5, 10, 8, 8, 22, 989164), params={'n_estimators': 709, 'learning_rate': 0.02070598529017565, 'num_leaves': 877, 'max_depth': 9, 'min_child_samples': 94, 'subsample': 0.5363259753060031, 'colsample_bytree': 0.9194824646782057}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'n_estimators': IntDistribution(high=1000, log=False, low=100, step=1), 'learning_rate': FloatDistribution(high=0.3, log=True, low=0.01, step=None), 'num_leaves': IntDistribution(high=3000, log=False, low=20, step=1), 'max_depth': IntDistribution(high=15, log=False, low=3, step=1), 'min_child_samples': IntDistribution(high=100, log=False, low=5, step=1), 'subsample': FloatDistribution(high=1.0, log=False, low=0.5, step=None), 'colsample_bytree': FloatDistribution(high=1.0, log=False, low=0.5, step=None)}, trial_id=32,

# Print best hyperparameters

In [21]:

best_params = study.best_params
best_params["objective"] = "regression_l1"
best_params["random_state"] = 42

print("Best hyperparameters:")
for k, v in best_params.items():
    print(f"{k}: {v}")

Best hyperparameters:
n_estimators: 709
learning_rate: 0.02070598529017565
num_leaves: 877
max_depth: 9
min_child_samples: 94
subsample: 0.5363259753060031
colsample_bytree: 0.9194824646782057
objective: regression_l1
random_state: 42


# Log hyperparameters to mlflow

In [23]:

# --- MLflow logging ---
with mlflow.start_run(run_name="lightgbm_optuna_tuning") as run:
    mlflow.log_param("model", "LightGBM")
    mlflow.log_param("n_lags", 672)
    for k, v in best_params.items():
        mlflow.log_param(k, v)

    print("View run at:")
    print(f"{mlflow.get_tracking_uri()}/#/experiments/{run.info.experiment_id}/runs/{run.info.run_id}/#/experiments/{run.info.experiment_id}/runs/{run.info.run_id}")


View run at:
https://dagshub.com/Vr1155/citibike.mlflow/#/experiments/3/runs/35fa6b25085148968261b4d1accfeb96/#/experiments/3/runs/35fa6b25085148968261b4d1accfeb96
🏃 View run lightgbm_optuna_tuning at: https://dagshub.com/Vr1155/citibike.mlflow/#/experiments/3/runs/35fa6b25085148968261b4d1accfeb96
🧪 View experiment at: https://dagshub.com/Vr1155/citibike.mlflow/#/experiments/3
